In [2]:
%pip install numpy
%pip install tensorflow
%pip install scikit-learn
%pip install matplotlib
%pip install scikit-image
%pip install imutils
%pip install opencv-python
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 122.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 174.1 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]━━━ 1/2 [xgboost]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import pandas as pd
import re
import math
from skimage import feature
from skimage.feature import hog
from imutils import paths
#from google.colab.patches import cv2_imshow
import os
from scipy.stats import skew, kurtosis

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier



import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, RandomFlip, RandomRotation, RandomZoom
from keras.callbacks import EarlyStopping
from keras import regularizers, Input
from keras.models import Model, load_model

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


2025-07-18 16:05:16.846521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 16:05:16.879468: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:05:17.152876: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:05:17.381358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752865517.560687  166317 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752865517.61

In [2]:
def cnn(input_shape=(32,32,3), num_classes=10, pool='flatten'):
  model = Sequential()

  #  CNN em “três estágios” de convolução + pooling para extrair 
  # representações progressivamente mais abstratas, e em seguida camadas 
  # densas para mapear essas representações em classificações.

  # blocos 1, 2, 3 para aprendizado da representacao

  # 1
  # Conv2D(32, (3,3), …): aprende 32 filtros 3×3 sobre a entrada; 
  # padding='same' mantém as dimensões espaciais (32×32).
  # activation='relu': aplica ReLU, que ajuda a modelar não-linearidades.
  # Dropout(0.3): “desliga” aleatoriamente 30% dos neurônios para evitar overfitting.
  # MaxPooling2D(2×2): reduz pela metade altura e largura (e.g. de 32×32 para 16×16), mantendo o filtro mais forte em cada região.
  
  model.add(Conv2D(32, (3,3), padding='same'
                      , input_shape=input_shape
                      , kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
  
  # BatchNormalization() após cada convolução, antes da ReLU: 
  # estabiliza a distribuição das ativações, acelera e regulariza o treino.
  model.add(BatchNormalization())
  # Activation('relu') separada da Conv2D, seguindo o padrão Conv→BN→ReLU.
  model.add(Activation('relu'))
  
  model.add(Conv2D(32, (3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))

  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # 2
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # 3
  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # camadas densas
  # parte totalmente conectada
  # Flatten(): “desenrola” o volume de ativação (por exemplo, 4×4×128 → 2048 neurônios).
  # GlobalMaxPooling2D(): toma o máximo de cada mapa de ativação, resultando em 128 valores fixos (um por filtro), independente da dimensão espacial.
  if pool == 'flatten': model.add(Flatten())
  else: model.add(GlobalMaxPooling2D())
  
  # Dense(512, relu): camada oculta com 512 neurônios para aprender combinações de características extraídas.
  model.add(Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)))
  model.add(Dropout(0.2))
  model.add(Dense(num_classes, activation='softmax',))

  return model

In [3]:
fp_weights = './checkpoints/bestmodel_cnn_basica_global_nadam.keras'


In [2]:
# 2) Carregue os dados CIFAR‑10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# normalize para [0,1]
x_train = x_train.astype('float32') / 255.0
x_test  = x_test .astype('float32') / 255.0
# achate os labels para vetor 1D
y_train = y_train.ravel()
y_test  = y_test.ravel()

In [5]:
model_cnn = cnn(input_shape=(32,32,3), num_classes=10, pool='global')
model_cnn.load_weights(fp_weights)

/home/philipe.hfransozi/Downloads/atividade_3/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-18 10:53:35.644950: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
cnn_extractor = Sequential(model_cnn.layers[:-2])


In [7]:
X_train_cnn = cnn_extractor.predict(x_train, batch_size=64, verbose=1)
X_test_cnn  = cnn_extractor.predict(x_test,  batch_size=64, verbose=1)

KeyboardInterrupt: 

In [8]:
print("Shape dos features (train):", X_train_cnn.shape)  # (50000, 512)
print("Shape dos features (test) :", X_test_cnn.shape)   # (10000, 512)

Shape dos features (train): (50000, 512)
Shape dos features (test) : (10000, 512)


In [9]:
ds_train = tf.data.Dataset.from_tensor_slices(x_train)
ds_test = tf.data.Dataset.from_tensor_slices(x_test)

In [11]:
W, H = 224, 224
def _pre(img):
    img = tf.image.resize(img, [W, H])
    img = preprocess_input(img)
    return img

In [12]:
batch_size = 64
ds_train = ds_train.map(_pre, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(_pre, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [13]:
mnet = MobileNetV2(
    include_top=False,
    weights='imagenet',
    pooling='avg',
    input_shape=(W, H, 3)
)

X_train_netv2 = mnet.predict(ds_train, batch_size=64, verbose=1)
X_test_netv2  = mnet.predict(ds_test,  batch_size=64, verbose=1)

print("X_train_feats.shape:", X_train_netv2.shape)  # (50000,1280)
print("X_test_feats.shape :", X_test_netv2.shape)   # (10000,1280)

782/782 ━━━━━━━━━━━━━━━━━━━━ 319s 405ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 63s 404ms/step
X_train_feats.shape: (50000, 1280)
X_test_feats.shape : (10000, 1280)


In [3]:
from tensorflow.keras.applications import (
    ResNet50, InceptionV3, Xception,
    resnet50, inception_v3, xception
)

In [4]:
def make_feature_extractor(model_name='resnet50'):
    if model_name == 'resnet50':
        ModelClass    = ResNet50
        preprocess_fn = resnet50.preprocess_input
        target_size   = (224, 224)
    elif model_name == 'inceptionv3':
        ModelClass    = InceptionV3
        preprocess_fn = inception_v3.preprocess_input
        target_size   = (299, 299)
    elif model_name == 'xception':
        ModelClass    = Xception
        preprocess_fn = xception.preprocess_input
        target_size   = (299, 299)
    else:
        raise ValueError(f"Unknown model {model_name}")

    base = ModelClass(
        include_top=False,
        weights='imagenet',
        pooling='avg',
        input_shape=(*target_size, 3)
    )
    return base, preprocess_fn, target_size

In [5]:
# Cria o tf.data pipeline que faz resize + preprocess_input certo
def make_dataset(x, batch_size=64):
    ds = tf.data.Dataset.from_tensor_slices(x)
    def _prep(img):
        img = tf.image.resize(img, [W, H])
        img = preprocess_fn(img)
        return img
    return ds.map(_prep, num_parallel_calls=tf.data.AUTOTUNE) \
             .batch(batch_size) \
             .prefetch(tf.data.AUTOTUNE)

In [6]:
backbone, preprocess_fn, (W, H) = make_feature_extractor('inceptionv3')

2025-07-18 16:06:57.294469: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:
ds_train_inceptionv3 = make_dataset(x_train)
ds_test_inceptionv3  = make_dataset(x_test)

# Extrai features sem encher a RAM
X_train_inceptionv3 = backbone.predict(ds_train_inceptionv3, verbose=1)
X_test_inceptionv3  = backbone.predict(ds_test_inceptionv3,  verbose=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 1647s 2s/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 365s 2s/step


In [7]:
backbone, preprocess_fn, (W, H) = make_feature_extractor('xception')
ds_train_xception = make_dataset(x_train)
ds_test_xception  = make_dataset(x_test)

# Extrai features sem encher a RAM
X_train_xception = backbone.predict(ds_train_xception, verbose=1)
X_test_xception  = backbone.predict(ds_test_xception,  verbose=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 1998s 3s/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 476s 3s/step


In [20]:
backbone, preprocess_fn, (W, H) = make_feature_extractor()
ds_train_resnet50 = make_dataset(x_train)
ds_test_resnet50  = make_dataset(x_test)

# Extrai features sem encher a RAM
X_train_resnet50 = backbone.predict(ds_train_resnet50, verbose=1)
X_test_resnet50  = backbone.predict(ds_test_resnet50,  verbose=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 832s 1s/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step


In [8]:
from sklearn.decomposition import PCA


In [9]:
# 1) Definição dos pipelines base
pipelines = {
    # 'SVC': Pipeline([
    #     ('scaler', StandardScaler()),
    #     ('pca', PCA()),
    #     ('clf', SVC(probability=True))
    # ]),
    'RandomForest': Pipeline([
        ('scaler', StandardScaler()), 
        ('pca',     PCA()),
        ('clf', RandomForestClassifier(random_state=42))
    ]),
    'MLPClassifier': Pipeline([
        ('scaler', StandardScaler()),
        ('pca',     PCA()),
        ('clf', MLPClassifier(random_state=42, max_iter=250))
    ]),
}


In [10]:
# 2) Grades de parâmetros para cada um
param_grids = {
    # 'SVC': {
    #     'clf__C': [1, 5, 10],
    #     'clf__gamma': [0.001, 0.0001, 'scale'],
    #     'clf__kernel': ['rbf', 'linear']
    # },
    'RandomForest': {
        'clf__n_estimators': [100, 200, 500],
        'clf__max_depth': [None, 10, 20],
        'clf__min_samples_split': [2, 5, 10]
    },
    'MLPClassifier': {
        'clf__hidden_layer_sizes': [(256,), (512,)],
        'clf__alpha': [1e-4, 1e-3, 1e-2],
        'clf__learning_rate_init': [1e-3, 1e-4],
        'clf__activation':['relu', 'logistic']
    }
}

param_grids_pca = {
    **param_grids,  # preserva os grids de clf__*
    **{            # adiciona pca__n_components para **todos** os pipelines
        name: {
            **params,
            'pca__n_components': [300]
        }
        for name, params in param_grids.items()
    }
}

In [11]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

def classificacao(X_train, X_test, y_train, y_test, param_grids, pipelines):
    # 1) Busca dos melhores estimadores
    best_estimators = {}
    for name, pipe in pipelines.items():
        print(f"\n--- Tuning {name} ---")
        grid = GridSearchCV(
            pipe,
            param_grid=param_grids[name],
            cv=3,
            scoring='accuracy',
            n_jobs=-1,
            verbose=1
        )
        grid.fit(X_train, y_train)
        print(f"Best params for {name}: {grid.best_params_}")
        print(f"Best CV accuracy for {name}: {grid.best_score_:.4f}")
        best_estimators[name] = grid.best_estimator_

    # 2) Avaliação no conjunto de teste
    resultados = []
    for name, est in best_estimators.items():
        y_pred = est.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        cm  = confusion_matrix(y_test, y_pred)
        
        # Armazena para o resumo
        resultados.append({
            'Modelo': name,
            'Acurácia': acc,
            'Matriz de Confusão': cm
        })
        
    # 3) Exibe tabela resumo (sem imprimir a matrix inteira na célula)
    df_resumo = pd.DataFrame(resultados)
    # Mostra as duas primeiras colunas
    print("\n=== Resumo de Acurácias ===")
    print(df_resumo[['Modelo','Acurácia']].to_string(index=False))

    # 4) Exibe cada matriz de confusão detalhadamente
    for row in resultados:
        name = row['Modelo']
        cm   = row['Matriz de Confusão']
        print(f"\nMatriz de Confusão para {name}:")
        print(pd.DataFrame(
            cm,
            index=[f"True_{i}" for i in range(cm.shape[0])],
            columns=[f"Pred_{i}" for i in range(cm.shape[1])]
        ).to_string())

In [ ]:
classificacao(X_train_cnn, X_test_cnn, y_train, y_test, param_grids_pca, pipelines)

In [ ]:
classificacao(X_train_netv2, X_test_netv2, y_train, y_test, param_grids_pca, pipelines)

In [ ]:
classificacao(X_train_inceptionv3, X_test_inceptionv3, y_train, y_test, param_grids_pca, pipelines)

In [ ]:
classificacao(X_train_xception, X_test_xception, y_train, y_test, param_grids_pca, pipelines)

In [ ]:
classificacao(X_train_resnet50, X_test_resnet50, y_train, y_test, param_grids_pca, pipelines)